In [25]:
import pandas as pd

# from data.kbo.statiz import KBOTeams
from data.kbo.naver import NaverGame
from data.kbo.naver import SingleInning

from datetime import datetime

In [26]:
import requests

KBO_NAVER_LOGS_URL = "https://api-gw.sports.naver.com/schedule/games"

def request_player_stats(home_team, away_team, date: datetime, inning: int):
    header = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "authority": "api-gw.sports.naver.com",
        "Origin": "https://m.sports.naver.com",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
    }
    gid = f"{date.strftime('%Y%m%d')}{home_team}{away_team}0{date.year}"
    url = f"{KBO_NAVER_LOGS_URL}/{gid}/relay"

    param = {'inning': inning}
    resp = requests.get(url, params=param, headers=header)

    if resp.status_code == 200:
        jstr = resp.json()

        return jstr['result']
    
    else:
        RuntimeError(resp.status_code)

In [27]:
test_log = request_player_stats("HT", "LT", datetime(2024, 9, 28), 1)

In [ ]:
test_log['textRelayData'].keys()

In [ ]:
[[i[k] for k in ['name', 'seqno', 'weight', 'height', 'pcode']] for i in test_log['textRelayData']['homeLineup']['pitcher']]

In [ ]:
[[i[k] for k in ['name', 'posName', 'seqno', 'batOrder', 'weight', 'height', 'pcode']] for i in test_log['textRelayData']['homeLineup']['batter']]

In [ ]:
[[i[k] for k in ['name', 'seqno', 'weight', 'height', 'pcode']] for i in test_log['textRelayData']['awayLineup']['pitcher']]

In [ ]:
[[i[k] for k in ['name', 'posName', 'seqno', 'batOrder', 'weight', 'height', 'pcode']] for i in test_log['textRelayData']['awayLineup']['batter']]

### Game itself

In [2]:
log = NaverGame("HT", "LT", datetime(2024, 9, 28))
d_ = log.data()

In [ ]:
inning_manager = SingleInning(1, "away")
xs, ys = list(), list()
for i, d in enumerate(d_):
    training = inning_manager.insert_log(d)
    if training is not None:
        x, y = training
        xs.append(x)
        ys.append(y)
    else:
        print(f"passing log {i}")
    
    if inning_manager.outs >= 3:
        inning_manager = inning_manager.advance_to_next()  # Change side or inning
        print(f"moved to inning {int(inning_manager.curr_inning)}, {inning_manager.home_away}")


In [8]:
total_logs = pd.concat([pd.concat(xs), pd.concat(ys)], axis=1).reset_index(drop=True)

In [ ]:
total_logs